# Code Generator

The requirement : use a Frontier model to generate high performance C++ code from Python code.

In [2]:
!pip install anthropic
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [3]:
#imports

import os
import io
import sys
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from google.colab import userdata

In [4]:
# initialize
openai_key = userdata.get('openapi')
anthropic_key = userdata.get('claude')

openai = OpenAI(api_key=openai_key)
claude = anthropic.Anthropic(api_key=anthropic_key)
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [5]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac."
system_message += "Response only with C++ code; use comments sparingly and provide any explanation other than occasional comments"
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [6]:
def user_prompt_for(python):
  user_prompt = "Rewrite this Python code in C++ with the fastest possible implmentation that produces identical output in the least time."
  user_prompt += "respond only with C++ code; explain your work and show the output as well"
  user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n" #this hint is required for gpt-4o
  user_prompt += python
  return user_prompt

In [7]:
def messages_for(python):
  return [
      {"role": "system", "content": system_message},
      {"role": "user", "content": user_prompt_for(python)}
  ]

In [9]:
# write to a file called optimized.cpp

def write_output(cpp):
  code = cpp.replace("```cpp", "").replace("```", "")
  with open("optimized.cpp", "w") as f:
    f.write(code)

In [8]:
def optimize_gpt(python):
  stream = openai.chat.completions.create(
      model=OPENAI_MODEL,
      messages=messages_for(python),
      stream=True
  )
  reply = ""
  for chunk in stream:
    fragment = chunk.choices[0].delta.content or ""
    reply += fragment
    print(fragment, end="", flush=True)
  write_output(reply)

In [10]:
def optimize_claude(python):
  result = claude.messages.stream(
      model=CLAUDE_MODEL,
      max_tokens=2000,
      system=system_message,
      messages=[{"role":"user", "content": user_prompt_for(python)}]
  )
  reply = ""
  with result as stream:
    for text in stream.text_stream:
      reply += text
      print(text, end="", flush=True)
  write_output(reply)

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
  result = 1.0
  for i in range(1, iterations+1):
    j = i * param1 - param2
    result -= (1/j)
    result += (1/j)
  return result

start_time = time.time()
result = calculate(100_000_000,4,1) * 4
end_time = time.time()

print(f"Result: {result: .12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi) #this runs python code inside the string.

Result:  4.000000000000
Execution Time: 36.466662 seconds


In [ ]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Function to perform calculation
double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    double j;
    for (long long i = 1; i <= iterations; ++i) {
        j = static_cast<double>(i * param1 - param2);
        // Although the operations are neutralized later, keeping them for identical functional logic
        result -= (1.0 / j);
        result += (1.0 / j);
    }
    return result;
}

int main() {
    using namespace std::chrono;

    auto start_time = high_resolution_clock::now();
    double result = calculate(100'000'000, 4, 1) * 4;
    auto end_time = high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12) 
              << "Result: " << result << std::endl;

    double execution_time = duration_cast<duration<double>>(end_time - start_time).count();
    std::cout << "Execution Time: " << std::setprecision(6) 
              << execution_time << " sec

In [ ]:
optimize_claude(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    #pragma omp parallel for reduction(+:result)
    for (long long i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i) * param1 - param2;
        result += (1.0 / j) - (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100'000'000, 4, 1) * 4;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    
    std::cout << "Result: " << std::fixed << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << duration.count() / 1e6 << " seconds" << std::endl;
    
    return 0;
}

// Compile with: g++ -O3 -fopenmp -march=native -mtune=native filename.cpp -o output
// Run with: ./output

Now to build Gradio

In [12]:
def stream_gpt(python):
  stream = openai.chat.completions.create(
      model=OPENAI_MODEL,
      messages=messages_for(python),
      stream=True
  )
  reply = ""
  for chunk in stream:
    fragment = chunk.choices[0].delta.content or ""
    reply += fragment
    yield reply

In [13]:
def stream_claude(python):
  result = claude.messages.stream(
      model=CLAUDE_MODEL,
      max_tokens=2000,
      system=system_message,
      messages=[{"role":"user", "content": user_prompt_for(python)}]
  )
  reply = ""
  with result as stream:
    for text in stream.text_stream:
      reply += text
      yield reply

In [ ]:
stream_claude(pi)

<generator object stream_claude at 0x7a4b380c5000>

In [14]:
def optimize(python,model):
  if model=="GPT":
    result = stream_gpt(python)
  elif model=="Claude":
    result = stream_claude(python)
  else:
    raise ValueError("Unknown model")
  for stream_so_far in result:
    yield stream_so_far.replace("```cpp", "").replace("```", "")

In [ ]:
with gr.Blocks() as ui:
  with gr.Row():
    python = gr.Textbox(label="Python code:", lines=10)
    cpp = gr.Textbox(label="C++ code", lines=10)
  with gr.Row():
    model = gr.Dropdown(["GPT","Claude"], label="Select model", value="GPT")
    convert = gr.Button("Convert code")

  convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a4cf61286ff3605db1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [15]:
import io
import sys

def execute_python(code):
    output = io.StringIO()
    sys_stdout = sys.stdout  # Save current stdout
    sys.stdout = output      # Redirect stdout to capture prints

    try:
        exec(code, {})       # Execute the provided code
    except Exception as e:
        print(f"Error: {e}")
    finally:
        sys.stdout = sys_stdout  # Restore original stdout

    return output.getvalue()


In [16]:
import subprocess

def write_output(code):
    with open("temp.cpp", "w") as f:
        f.write(code)

def execute_cpp(code):
    write_output(code)
    try:
        # Compile
        compile_cmd = ["clang++", "temp.cpp", "-O2", "-std=c++17", "-march=native", "-o", "optimized.exe"]
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True)

        if compile_result.returncode != 0:
            return f"Compilation error:\n{compile_result.stderr}"

        # Run
        run_cmd = ["optimized.exe"] if sys.platform == "win32" else ["./optimized.exe"]
        run_result = subprocess.run(run_cmd, capture_output=True, text=True, timeout=5)

        return run_result.stdout or run_result.stderr

    except subprocess.CalledProcessError as e:
        return f"Runtime error:\n{e.stderr}"
    except Exception as e:
        return f"Unexpected error:\n{str(e)}"


In [17]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050}
 """

In [ ]:
with gr.Blocks(css=css) as ui:
  gr.Markdown("## Convert code from Python to C++")
  with gr.Row():
    python = gr.Textbox(label="Python code:",value=pi, lines=10)
    cpp = gr.Textbox(label="C++ code", lines=10)
  with gr.Row():
    model = gr.Dropdown(["GPT","Claude"], label="Select model", value="GPT")
  with gr.Row():
    convert = gr.Button("Convert code")
  with gr.Row():
    python_run = gr.Button("Run Python")
    cpp_run = gr.Button("Run C++")
  with gr.Row():
    python_output = gr.TextArea(label="Python output", elem_classes=["python"])
    cpp_output = gr.TextArea(label="C++ output",elem_classes=["cpp"])

  convert.click(optimize, inputs=[python, model], outputs=[cpp])
  python_run.click(execute_python, inputs=[python], outputs=[python_output])
  cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_output])

ui.launch(inbrowser=True)


# Day-4

In [1]:
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [21]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [22]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_gemma = "google/codegemma-7b-it"
CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(code_qwen)
messages = messages_for(pi)
text = tokenizer.apply_chat_template(messages,tokenize=False, add_generation_prompt=True)

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

In [ ]:
client = InferenceClient(CODE_QWEN_URL, token=hf_token) # need to have my url with hf token permission to the model.
response = client.text_generation(text, stream=True, details=True,max_new_tokens=3000)
for r in response:
  print(r.token.text, end="")

In [28]:
def stream_code_qwen(python):
  tokenizer = AutoTokenizer.from_pretrained(code_qwen)
  messages = messages_for(python)
  text = tokenizer.apply_chat_template(messages,tokenize=False, add_generation_prompt=True)
  client = InferenceClient(CODE_QWEN_URL, token=hf_token)
  response = client.text_generation(text, stream=True, details=True,max_new_tokens=3000)
  result = ""
  for r in response:
    result += r.token.text
    yield result

In [31]:
def optimize(python,model):
  if model=="GPT":
    result = stream_gpt(python)
  elif model=="Claude":
    result = stream_claude(python)
  elif model=="CodeQwen":
    result = stream_code_qwen(python)
  else:
    raise ValueError("Unknown model")
  for stream_so_far in result:
    yield stream_so_far

In [ ]:
with gr.Blocks(css=css) as ui:
  gr.Markdown("## Convert code from Python to C++")
  with gr.Row():
    python = gr.Textbox(label="Python code:",value=pi, lines=10)
    cpp = gr.Textbox(label="C++ code", lines=10)
  with gr.Row():
    model = gr.Dropdown(["GPT","Claude","CodeQwen"], label="Select model", value="GPT")
  with gr.Row():
    convert = gr.Button("Convert code")
  with gr.Row():
    python_run = gr.Button("Run Python")
    cpp_run = gr.Button("Run C++")
  with gr.Row():
    python_output = gr.TextArea(label="Python output", elem_classes=["python"])
    cpp_output = gr.TextArea(label="C++ output",elem_classes=["cpp"])

  convert.click(optimize, inputs=[python, model], outputs=[cpp])
  python_run.click(execute_python, inputs=[python], outputs=[python_output])
  cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_output])

ui.launch(inbrowser=True)